In [2]:
from scipy.integrate import odeint
from mpl_toolkits import mplot3d
from numpy import sin, cos, pi, linspace, abs
from matplotlib.pyplot import plot, show, figure, axes, scatter, xlabel, ylabel, legend, title, subplots, gca
import PyQt5 
%matplotlib qt

def dp_rhs(v, t, p):
    """
    Defines the differential equations for the double pendulum system.

    Parameters:
        v :  vector of the state variables:
                  v = [p1,w1,p2,w2]
        t :  time
        p :  vector of the parameters:
                  p = [m1,m2,l1,l2,g]
    
    Return:
        f :  list of the dp_rhs equations for the 4 ODES
    """
    p1, w1, p2, w2 = v
    m1, m2, l1, l2, g = p

    # Create f = (p1'=,w1'=,p2'=,w2'=):

    w1_dot = (-(m1+m2)*g*sin(p1)-m2*l2*w2*w2*sin(p1-p2)-m2*cos(p1-p2)*l1*w1*w1*sin(p1-p2)+m2*g*cos(p1-p2)*sin(p2))/((m1+m2)*l1-m2*l1*(cos(p1-p2)*cos(p1-p2)))
    w2_dot = (l1*w1*w1*2*sin(p1-p2)-g*sin(p2)-l1*w1_dot*cos(p1-p2))/l2

    f = [w1,w1_dot,w2,w2_dot]
    return f

def sa_dp_rhs(v, t, p):
    """
    Defines the differential equations for the small angle approximation of the double pendulum system.

    Parameters:
        v :  vector of the state variables:
                  v = [p1,w1,p2,w2]
        t :  time
        p :  vector of the parameters:
                  p = [m1,m2,l1,l2,g]
    
    Return:
        f :  list of the sa_dp_rhs equations for the 4 ODES
    """
    p1, w1, p2, w2 = v
    m1, m2, l1, l2, g = p

    # Create f = (p1'=,w1'=,p2'=,w2'=):

    w1_dot = (g*l1)*(m2*p2-(m1+m2)*p1)/(m1*l1*l1)
    w2_dot = -(g*p2+l1*w1_dot)/l2

    f = [w1,w1_dot,w2,w2_dot]
    return f

# Parameter values
# Default Masses:
m1 = 1.0
m2 = 1.0
# gravity:
g = 9.8
# Default lengths
l1 = 1.0
l2 = 1.0

# Initial conditions
p1 = pi/4   # (\phi_1) just for testing
w1 = 0.0    # (\omega_1) just for testing
p2 = pi/4   # (\phi_2) just for testing
w2 = 0.0    # (\omega_2) just for testing
p = [m1, m2, l1, l2, g] # Parameters
IC = [p1,0,p2,0] # Default initial conditions for plotting 
N = 100
t_end = pi/2 # Kept small so that we don't get the angular velocity of the mass swinging back towards vertical
t_interval = linspace(0, t_end, N)

# Error tolerance
abserr = 1.0e-8
relerr = 1.0e-6

# Functions for graphing ----------------------------------------------------------------------------------------------------------------------------

def find_vertical(l1, l2, p1, p2):
    """
    Find the index of w2 when m2 reaches the vertical.
    
    Parameters:
        l1 :  length of first string
        l2 :  length of second string
        p1 :  array of the first angle's possible values
        p2 :  array of the second angle's possible values
    """
    closest = 999
    closest_index = None
    for i in range(len(p1)):
        x = abs(l1*sin(p1[i])+l2*sin(p2[i]))
        if x < closest:
            closest = x
            closest_index = i
    return closest_index

def p_w_plot(rhs, w_val=3, show_level_curve=False, iter_n=25, w_err=0.05, angle_start=pi/15, angle_end=pi/2):
    """
    Generate the 3D plot for varying angles and the corresponding w2.

    Parameter:
        rhs :  rhs of ODE to be used for odeint()

    Optional Parameters:
        w_val :  Level curve value
        iter_n :  number of iterations for getting points of the angles
        w_err :  allowed error in getting the level curve of w2 (similar w2 values)
        show_level_curve :  graphs the level curve if true
        angle_start :  starting angle when iterating angles
    """
    fig = figure()
    ax = axes(projection='3d')

    p1_err_list = []
    p2_err_list = []
    w2_err_list = []

    p1_list = []
    p2_list = []
    w2_list = []

    max_p2_list = []
    max_w2_list = []

    min_p2_list = []
    min_w2_list = []

    iter_p1 = linspace(angle_start,angle_end, iter_n)
    iter_p2 = linspace(angle_start,angle_end, iter_n)

    for p1 in iter_p1:
        max_w2_for_current_p1 = 0
        min_w2_for_current_p1 = 9999
        for p2 in iter_p2:
            ic = [p1, w1, p2, w2]
            v_sol = odeint(rhs, ic, t_interval, args=(p,), atol=abserr, rtol=relerr)
            p1_arr, p2_arr, w2_arr = v_sol[:,0],v_sol[:,2], v_sol[:,3]
            t_index = find_vertical(l1,l2,p1_arr,p2_arr)
            w2_at_vertical = -w2_arr[t_index] # negative for correcting sign of w2 (want positive)

            if w2_at_vertical > max_w2_for_current_p1:
                max_w2_for_current_p1 = w2_at_vertical
            if w2_at_vertical < min_w2_for_current_p1:
                min_w2_for_current_p1 = w2_at_vertical

            w2_list.append(w2_at_vertical)
            p1_list.append(p1)
            p2_list.append(p2)
            if show_level_curve:
                if abs(w_val - w2_at_vertical) < w_err:
                    p1_err_list.append(p1)
                    p2_err_list.append(p2)
                    w2_err_list.append(w2_at_vertical)
        max_w2_list.append(max_w2_for_current_p1)
        max_p2_list.append(p2_list[w2_list.index(max_w2_for_current_p1)])
        min_w2_list.append(min_w2_for_current_p1)
        min_p2_list.append(p2_list[w2_list.index(min_w2_for_current_p1)])

    # Plot of the 2 angles and w2
    if not show_level_curve:
        ax.plot_trisurf(p1_list,p2_list,w2_list,cmap='viridis',edgecolor='green', zorder=1)
        ax.plot(iter_p1, max_p2_list, max_w2_list, zorder=4, color="red", label='biggest $\omega_2$ for a given $\phi_1$')
        ax.plot(iter_p1, min_p2_list, min_w2_list, zorder=4, color="blue", label='smallest $\omega_2$ for a given $\phi_1$')
        title("Angular Velocity ($\omega_2$) vs $\phi_1$ and $\phi_2$")
        legend()

    if show_level_curve: 
        # Surface curve for the values of p1, p2 with the same w2's.
        ax.scatter3D(p1_err_list,p2_err_list,w2_err_list,color="purple")
        ax.set_xlim(0, angle_end)
        ax.set_ylim(0, angle_end)
        ax.set_zlim(w_val-0.5, w_val+0.5)
        title(f"Level \"Curve\" of $\omega_2$ vs $\phi_1$ and $\phi_2$ around $\omega_2 = $ {w_val}")

    ax.set_xlabel("$\phi_1$")
    ax.set_ylabel("$\phi_2$")
    ax.set_zlabel("$\omega_2$")

def length_plot(rhs, plot_l1=True, n=50, l_max=10):
    """
    Generate the 2D plot for the varying lengths and the corresponding w2.

    Parameter:
        rhs :  rhs of ODE to be used for odeint()

    Optional Parameters:
        n :  number of iterations for getting points of the lengths
        plot_l1 :  plots graph of l1 vs w2 if true, plots l2 vs w2 otherwise
        l_max :  maximum length when plotting
    """
    l_arr = linspace(0.25,l_max, n)
    p_l = p.copy()
    w2_list = []
    for l in l_arr:
        if plot_l1:
            p_l[2] = l
        else:
            p_l[3] = l
        v_sol = odeint(rhs, IC, t_interval, args=(p_l,), atol=abserr, rtol=relerr)
        p1_arr, p2_arr, w2_arr = v_sol[:,0],v_sol[:,2], v_sol[:,3]
        if plot_l1:
            t_index = find_vertical(l,l2,p1_arr,p2_arr)
        else:
            t_index = find_vertical(l1,l,p1_arr,p2_arr)
        w2_list.append(-w2_arr[t_index])
    plot(l_arr,w2_list,'-o',label='l1' if plot_l1 else 'l2')
    legend(loc="best")
    
    if plot_l1:
        xlabel("$l_1$")
        title("Angular Velocity ($\omega_2$) vs $l_1$")
    else:
        xlabel("$l_2$")
        title("Angular Velocity ($\omega_2$) vs $l_2$")
    ylabel("$\omega_2$")

def mass_plot(rhs, plot_m1=True, n=50, m_max=50):
    """
    Generate the 2D plot for the varying masses and the corresponding w2.

    Parameter:
        rhs :  rhs of ODE to be used for odeint()

    Optional Parameters:
        n :  number of iterations for getting points of the masses
        plot_m1 :  plots graph of m1 vs w2 if true, plots m2 vs w2 otherwise
        m_max :  maximum mass when plotting
    """
    m_arr = linspace(0.25,m_max, n)
    p_m = p.copy()
    w2_list = []
    for m in m_arr:
        if plot_m1:
            p_m[0] = m
        else:
            p_m[1] = m
        v_sol = odeint(rhs, IC, t_interval, args=(p_m,), atol=abserr, rtol=relerr)
        p1_arr, p2_arr, w2_arr = v_sol[:,0],v_sol[:,2], v_sol[:,3]
        t_index = find_vertical(l1,l2,p1_arr,p2_arr)
        w2_list.append(-w2_arr[t_index])
    plot(m_arr,w2_list,'-o',label='m1' if plot_m1 else 'm2')
    legend(loc="best")
    
    if plot_m1:
        xlabel("$m_1$")
        title("Angular Velocity ($\omega_2$) vs $m_1$")
    else:
        xlabel("$m_2$")
        title("Angular Velocity ($\omega_2$) vs $m_2$")
    ylabel("$\omega_2$")

def plot_all_length(rhs, n=50,l_max=10):
    """
    Plot all 2 graphs of lengths vs w2.
    
    Parameter:
        rhs :  rhs of ODE to be used for odeint()

    Optional Parameters:
        n :  number of iterations for getting points of the lengths
        l_max :  maximum length when plotting
    """
    length_plot(rhs, plot_l1=False,n=n,l_max=l_max)
    length_plot(rhs, n=n,l_max=l_max)
    xlabel("lengths")
    title("Angular Velocity ($\omega_2$) vs $l_1$ and $l_2$")

def plot_all_mass(rhs, n=50,m_max=50):    
    """
    Plot all 2 graphs of masses vs w2.
    
    Parameter:
        rhs :  rhs of ODE to be used for odeint()

    Optional Parameters:
        n :  number of iterations for getting points of the masses
        m_max :  maximum mass when plotting
    """
    mass_plot(rhs, plot_m1=False,n=n,m_max=m_max)
    mass_plot(rhs, n=n,m_max=m_max)
    xlabel("masses")
    title("Angular Velocity ($\omega_2$) vs $m_1$ and $m_2$")

def plot_all_p_w(rhs, w_val=3.5, iter_n=30, angle_start=pi/15, angle_end=pi/2):
    """
    Plot (separately) both graphs of angles vs w2 and the level curve of w2.

    Parameter:
        rhs :  rhs of ODE to be used for odeint()

    Optional Parameters:
        w_val :  Level curve value
        iter_n :  number of iterations for getting points of the angles
    """
    p_w_plot(rhs, w_val=w_val,iter_n=iter_n, angle_start=angle_start, angle_end=angle_end)
    p_w_plot(rhs, w_val=w_val,iter_n=iter_n,show_level_curve=True, angle_start=angle_start, angle_end=angle_end)

def plot_comparison(ic=[pi/2,0,pi/2,0], compare_p1=False, compare_p2=False, compare_differences_in_differences=False, \
                    plot_p_3D=False, plot_sa_p_3D=False, compare_p_3D=False, compare_p_3D_difference=False, \
                    plot_w_3D=False, plot_sa_w_3D=False, compare_w_3D=False, compare_w_3D_difference=False, \
                    compare_w1=False, compare_w2=False, compare_phase_space_p1=False, compare_phase_space_p2=False, \
                    show_periodicity=False, n=8, k=2):
    """
    Plot the comparison graphs for the corresponding boolean values. 

    Optional Parameters:
        ic :  initial conditions for solving the ODE
        n :  parameter for general plotting
        k :  parameter for general plotting (replace with n in the future)
        For the following optional paramters, if they are set to true, they will plot their descriptions
            compare_p1 :  plot the comparison graphs for p1(t) and small angle approximation of p1(t) and their differences
            compare_p2 :  plot the comparison graphs for p2(t) and small angle approximation of p2(t) and their differences
            compare_differences_in_differences :  plot the differences in p1 and p2 together as well as the differnces in their differences
            plot_p_3D :  plot time vs p1, p2
            plot_sa_p_3D :  plot time vs small angle p1, p2
            compare_p_3D :  plot comparison graph of time vs p1, p2 and small angle p1, p2 together
            compare_p_3D_difference :  plot comparison graph of time vs (p1 - small angle p1), (p2 - small angle p2)
            plot_w_3D :  plot time vs w1, w2
            plot_sa_w_3D :  plot time vs small angle w1, w2
            compare_w_3D :  plot comparison graph of time vs w1, w2 and small angle w1, w2 together
            compare_w_3D_difference :  plot comparison graph of time vs (w1 - small angle w1), (w2 - small angle w2)
            compare_w1 :  plot the comparison graphs for w1(t) and small angle approximation of w1(t) and their differences
            compare_w2 :  plot the comparison graphs for w2(t) and small angle approximation of w2(t) and their differences
            compare_phase_space_p1 :  plot the phase space of p1, w1
            compare_phase_space_p2 :  plot the phase space of p2, w2     
            show_periodicity :  increases the time inverval of the graph enough to show periodicity in differences
    """
    if show_periodicity or compare_p_3D_difference or compare_w_3D_difference:
        n *= 16
        k *= 8
    plotting3D = plot_p_3D or plot_sa_p_3D or compare_p_3D or plot_w_3D or plot_sa_w_3D or compare_w_3D 
    plot_phase_space = compare_phase_space_p2 or compare_phase_space_p1                        
    t = linspace(0, 2*k*pi/2 if plotting3D else k*pi if plot_phase_space else n*pi , \
                 2*k*N if plotting3D else 2*n*N)
    v_sol = odeint(dp_rhs, IC, t, args=(p,), atol=abserr, rtol=relerr)
    p1_arr, w1_arr, p2_arr, w2_arr = v_sol[:,0],v_sol[:,1],v_sol[:,2], v_sol[:,3]
    sa_v_sol = odeint(sa_dp_rhs, IC, t, args=(p,), atol=abserr, rtol=relerr)
    sa_p1_arr, sa_w1_arr, sa_p2_arr, sa_w2_arr = sa_v_sol[:,0],sa_v_sol[:,1],sa_v_sol[:,2], sa_v_sol[:,3]

    if compare_p1:
        fig, (ax1, ax2) = subplots(2)
        ax1.plot(t, p1_arr, label='$\phi_1$')
        ax1.plot(t, sa_p1_arr, label='$\Phi_1$')
        ax1.legend(loc="best")
        ax1.set_xlabel("t")
        ax1.set_ylabel("$\phi, \Phi_1$")
        ax1.set_title("$\phi_1, \Phi_1$ vs time")
        ax2.plot(t, p1_arr-sa_p1_arr)
        ax2.set_xlabel("t")
        ax2.set_ylabel("$\Phi_1$")
        ax2.set_title('Difference in $\phi_1, \Phi_1$')
        fig.tight_layout()

    if compare_p2:
        fig, (ax1, ax2) = subplots(2)
        ax1.plot(t, p2_arr, label='$\phi_2$')
        ax1.plot(t, sa_p2_arr, label='$\Phi_2$')
        ax1.legend(loc="best")
        ax1.set_xlabel("t")
        ax1.set_ylabel("$\phi_2, \Phi_2$")
        ax1.set_title("$\phi_2, \Phi_2$ vs time")
        ax2.plot(t, p2_arr-sa_p2_arr)
        ax2.set_xlabel("t")
        ax2.set_ylabel("$\phi_2, \Phi_2$")
        ax2.set_title('Difference in $\phi_2, \Phi_2$')
        fig.tight_layout()

    if compare_differences_in_differences:
        fig, (ax1, ax2) = subplots(2)
        ax1.plot(t, p1_arr-sa_p1_arr, label='difference in $\phi_1, \Phi_1$')
        ax1.plot(t, p2_arr-sa_p2_arr, label='difference in $\phi_2, \Phi_2$')
        ax1.set_xlabel("t")
        ax1.set_ylabel("$\phi$")
        ax1.set_title("Differences in $\phi_1, \Phi_1$ and $\phi_2, \Phi_2$, vs time")
        ax1.legend(loc="best")
        ax2.plot(t, (p2_arr-sa_p2_arr)-(p1_arr-sa_p1_arr))
        ax2.set_xlabel("t")
        ax2.set_ylabel("$\phi, \Phi$")
        ax2.set_title('Difference in the 2 Differences in $\phi_1, \Phi_1, \phi_2, \Phi_2$')
        fig.tight_layout()

    if plot_p_3D:
        fig = figure()
        ax = axes(projection='3d')
        ax.scatter3D(p1_arr,p2_arr,t,cmap='viridis',edgecolor='green', c=p1_arr)
        ax.set_xlabel("$\phi_1$")
        ax.set_ylabel("$\phi_2$")
        ax.set_zlabel("$t$")
        title("Time vs $\phi_1$ and $\phi_2$")

    if plot_sa_p_3D:
        fig = figure()
        ax = axes(projection='3d')
        ax.scatter3D(sa_p1_arr,sa_p2_arr,t,cmap='gnuplot',edgecolor='blue',c=sa_p1_arr)
        ax.set_xlabel("$\Phi_1$")
        ax.set_ylabel("$\Phi_2$")
        ax.set_zlabel("$t$")
        title("Time vs $\Phi_1$ and $\Phi_2$")

    if compare_p_3D:
        fig = figure()
        ax = axes(projection='3d')
        ax.scatter3D(p1_arr,p2_arr,t,cmap='viridis',edgecolor='green', zorder=1, label='t vs $\phi_1$, $\phi_2$',c=p1_arr)
        ax.scatter3D(sa_p1_arr,sa_p2_arr,t,cmap='gnuplot',edgecolor='blue', zorder=1, label='t vs $\Phi_1$, $\Phi_2$',c=sa_p1_arr)
        ax.set_xlabel("$\phi_1, \Phi_1$")
        ax.set_ylabel("$\phi_2, \Phi_2$")
        ax.set_zlabel("$t$")
        ax.legend()
        title("Comparison Graph of Time vs $\phi_1$, $\phi_2$ and $\Phi_1$, $\Phi_2$")

    if compare_p_3D_difference:
        fig = figure()
        ax = axes(projection='3d')
        ax.scatter3D(p1_arr - sa_p1_arr, p2_arr - sa_p2_arr,t, c=p1_arr - sa_p1_arr, cmap='ocean',edgecolor='blue')
        ax.set_xlabel("$\phi_1 - \Phi_1$")
        ax.set_ylabel("$\phi_2 - \Phi_2$")
        ax.set_zlabel("$t$")
        title("Comparison Graph of Time vs difference in $\phi_1$, $\phi_2$ and $\Phi_1$, $\Phi_2$")

    if plot_w_3D:
        fig = figure()
        ax = axes(projection='3d')
        ax.scatter3D(w1_arr,w2_arr,t,cmap='viridis',edgecolor='green',c=w1_arr)
        ax.set_xlabel("$\omega_1$")
        ax.set_ylabel("$\omega_2$")
        ax.set_zlabel("$t$")
        title("Time vs $\omega_1$ and $\omega_2$")

    if plot_sa_w_3D:
        fig = figure()
        ax = axes(projection='3d')
        ax.scatter3D(sa_w1_arr,sa_w2_arr,t,cmap='gnuplot',edgecolor='blue',c=sa_w1_arr)
        ax.set_xlabel("$\Omega_1$")
        ax.set_ylabel("$\Omega_2$")
        ax.set_zlabel("$t$")
        title("Time vs $\Omega_1$ and $\Omega_2$")

    if compare_w_3D:
        fig = figure()
        ax = axes(projection='3d')
        ax.scatter3D(w1_arr,w2_arr,t,cmap='viridis',edgecolor='green', zorder=1, label='t vs $\omega_1$, $\omega_2$',c=w1_arr)
        ax.scatter3D(sa_w1_arr,sa_w2_arr,t,cmap='gnuplot',edgecolor='blue', zorder=1, label='t vs $\Omega_1$, $\Omega_2$',c=sa_w1_arr)
        ax.set_xlabel("$\omega_1, \Omega_1$")
        ax.set_ylabel("$\omega_2, \Omega_2$")
        ax.set_zlabel("$t$")
        ax.legend()
        title("Comparison Graph of Time vs $\omega_1$, $\omega_2$ and $\Omega_1$, $\Omega_2$")

    if compare_w_3D_difference:
        fig = figure()
        ax = axes(projection='3d')
        ax.scatter3D(w1_arr - sa_w1_arr, w2_arr - sa_w2_arr,t, c=p1_arr - sa_p1_arr, cmap='ocean',edgecolor='blue')
        ax.set_xlabel("$\omega_1 - \Omega_1$")
        ax.set_ylabel("$\omega_2 - \Omega_2$")
        ax.set_zlabel("$t$")
        title("Comparison Graph of Time vs difference in $\omega_1$, $\omega_2$ and $\Omega_1$, $\Omega_2$")

    if compare_w1:
        fig, (ax1, ax2) = subplots(2)
        ax1.plot(t, w1_arr, label='$\omega_1$')
        ax1.plot(t, sa_w1_arr, label='$\Omega_1$')
        ax1.set_xlabel("t")
        ax1.set_ylabel("$\omega, \Omega$")
        ax1.set_title("$\omega_1, \Omega_1$ vs time")
        ax1.legend(loc="best")
        ax2.plot(t, w1_arr-sa_w1_arr,label='Difference in $\omega_1, \Omega_1$')
        ax2.set_xlabel("t")
        ax2.set_ylabel("$\omega, \Omega$")
        ax2.set_title('Difference in $\omega_1, \Omega_1$')
        ax2.legend(loc="best")
        fig.tight_layout()

    if compare_w2:
        fig, (ax1, ax2) = subplots(2)
        ax1.plot(t, w2_arr, label='$\omega_2$')
        ax1.plot(t, sa_w2_arr, label='$\Omega_2$')
        ax1.set_xlabel("t")
        ax1.set_ylabel("$\omega, \Omega$")
        ax1.set_title("$\omega_2, \Omega_2$ vs time")
        ax1.legend(loc="best")
        ax2.plot(t, w2_arr-sa_w2_arr, label='Difference in $\omega_2, \Omega_2$')
        ax2.legend(loc="best")
        ax2.set_xlabel("t")
        ax2.set_ylabel("$\omega, \Omega$")
        ax2.set_title('Difference in $\omega_2, \Omega_2$')
        fig.tight_layout()

    if compare_phase_space_p1:
        plot(p1_arr, w1_arr, label='phase space of $\phi_1, \omega_1$', zorder=1)
        plot(sa_p1_arr, sa_w1_arr, label='phase space of $\Phi_1, \Omega_1$', zorder=2)
        sa_path_start = [sa_p1_arr[0], sa_p1_arr[-1]]
        sa_path_end = [sa_w1_arr[0], sa_w1_arr[-1]]
        scatter(sa_path_start, sa_path_end, color="red", zorder=3)
        path_start = [p1_arr[0], p1_arr[-1]]
        path_end = [w1_arr[0], w1_arr[-1]]
        scatter(path_start, path_end, color="green", zorder=3)
        # scatter(sa_p1_arr[0::100], sa_w1_arr[0::100], color="orange")
        # scatter(p1_arr[0::10], w1_arr[0::10], color="blue")
        xlabel("$\phi_1, \Phi_1$")
        ylabel("$\omega_1, \Omega_1$")
        title("Phase Space")
        legend(loc="best")
    
    if compare_phase_space_p2:
        plot(p2_arr, w2_arr, label='phase space of $\phi_2, \omega_2$')
        plot(sa_p2_arr, sa_w2_arr, label='phase space $\Phi_2, \Omega_2$')
        sa_path_start = [sa_p2_arr[0], sa_p2_arr[-1]]
        sa_path_end = [sa_w2_arr[0], sa_w2_arr[-1]]
        scatter(sa_path_start, sa_path_end, color="red", zorder=3)
        path_start = [p2_arr[0], p2_arr[-1]]
        path_end = [w2_arr[0], w2_arr[-1]]
        scatter(path_start, path_end, color="green", zorder=3)
        xlabel("$\phi_2,\Phi_2$")
        ylabel("$\omega_2,\Omega_2$")
        title("Phase Space")
        legend(loc="best")


# Individual Plots:
# -----------------

# Double pendulum
# length_plot(dp_rhs,plot_l1=False)
# length_plot(dp_rhs)
# mass_plot(dp_rhs,plot_m1=False)
# mass_plot(dp_rhs)
# p_w_plot(dp_rhs)
# p_w_plot(dp_rhs, w_val=4,show_level_curve=True)

# Small angle approximation for the double pendulum
# length_plot(sa_dp_rhs,plot_l1=False)
# length_plot(sa_dp_rhs)
# mass_plot(sa_dp_rhs,plot_m1=False)
# mass_plot(sa_dp_rhs)
# p_w_plot(sa_dp_rhs)
# p_w_plot(sa_dp_rhs, w_val=4,show_level_curve=True)


# Plot all together:
# ------------------

# Double pendulum
# plot_all_length(dp_rhs, n=100,l_max=100)
# plot_all_mass(dp_rhs, n=100,m_max=100)
# plot_all_p_w(dp_rhs, w_val=2, iter_n=25)

# Small angle approximation for the double pendulum
# plot_all_length(sa_dp_rhs, n=100,l_max=100)
# plot_all_mass(sa_dp_rhs, n=100,m_max=100)
# plot_all_p_w(sa_dp_rhs, w_val=2, iter_n=25,angle_start=pi/64, angle_end=pi/4)

# Plot comparison graphs:
# -----------------------

plot_comparison(ic=[pi/2,0,pi/2,0], \
                compare_p1=1, compare_p2=1, compare_differences_in_differences=0, \
                plot_p_3D=0, plot_sa_p_3D=0, compare_p_3D=0, compare_p_3D_difference=0, \
                plot_w_3D=0, plot_sa_w_3D=0, compare_w_3D=0, compare_w_3D_difference=0, \
                compare_w1=0, compare_w2=0, compare_phase_space_p1=0, compare_phase_space_p2=0, \
                show_periodicity=0)

Observations:

Plotting against the two angles:
- lower values of p1 is required for the same angular velocity achieved compared to p2 (better to increase the first angle for optimal swing)
- general trend of favoring p1 for greater w2
- new plot: red

Plotting against the two lengths:
- w2 appraoches 0 (conservation of angular momentum)
- both graphs' w2 reaches maximum at about lengths between 0.4 to 0.7
- both graphs are similar and decreases exponentially (or like f(x)=1/x^n, n>0)

Plotting against the two masses:
- in w2(m1), w2 quickly approaches a value exponentially (and plateaus at w2 = 3.4)
    - at m1 = 37.5, w2 suddenly drops by about 0.14 (weird) 
- w2(m2) looks like an underdamped oscillator! (approaches w2 = 1.7)


Comparison graphs: (do later)

$\phi_1$:
- the graph bears similarity to the linear and nonlinear equations for a single pendulum (pseudo-sinusoidal)
- graphs differ after the start, but the difference is actually periodic with T = 125s

$\phi_2$
- similar analysis

compare_differences_in_differences
- same period of T = 125

$(\phi_1, \phi_2, t)$
- the rate of increase seems to be similar for the two angles as they graph lies about the plane y = x

small angle $(\phi_1, \phi_2, t)$
- similar to the actual dp ODE graphs

compare3D
- as you can see, the two graphs overlay each other closely, but not entirely the same

compare3D_difference
- period of T = 125 and converge at (0,0)
- the differnce graph also roughly lies on the plane y = x

compare_w1
- similar at the start, but goes out of sync quickly
- same period as others!

compare_w2
- similar graph to w1

phase space 1
- the ODE without small angle approx. has an almost closed trajectory, whereas the SA approx eqns clearly do not (interesting)
- this shows that the path of the double pendulum is non-deterministic, and chaotic in nature

phase space 2
- similar to phase space 1
